# Simulators

### Overview

**Simulator** maintains an instance of **Renderer** and **PhysicsEngine** and provides APIs to import **Scene**, **Object** and **Robot** into both of them and keep them synchronized at all time.

Some key functions are the following:
- `load`: initialize PyBullet physics engine and MeshRenderer
- `import_{scene, ig_scene}`: import the scene into PyBullet by calling `scene.load`, and then import it into MeshRenderer by calling `self.renderer.add_instance`. If `InteractiveIndoorScene` is imported using `import_ig_scene`, all objects in the scene are also imported.
- `import_{object, articulated_object, robot}`: import the object, articulated object and robot into the simulator in a similar manner
- `sync`: synchronize the poses of the dynamic objects (including the robots) between PyBullet and MeshRenderer. Specifically, it calls `update_position` for each object, in which it retrieve the object's pose in PyBullet, and then update its pose accordingly in MeshRenderer.

If `Simulator` uses `gui` mode, by default it will also maintain a `Viewer`, which essentially is a virtual camera in the scene that can render images. More info about the `Viewer` can be found here: [gibson2/render/viewer.py](https://github.com/StanfordVL/iGibson/blob/master/gibson2/render/viewer.py). Notably, you can press `h` in the `ExternalView` window to show the help menu for mouse/keyboard control.

Most of the code can be found here: [gibson2/simulator.py](https://github.com/StanfordVL/iGibson/blob/master/gibson2/simulator.py).



### Examples
In this example, we import a `StaticIndoorScene`, a `Turtlebot`, and ten `YCBObject` into the simulator. The code can be found here: [gibson2/examples/demo/simulator_example.py](https://github.com/StanfordVL/iGibson/blob/master/gibson2/examples/demo/simulator_example.py)

In [1]:
from gibson2.robots.turtlebot_robot import Turtlebot
from gibson2.simulator import Simulator
from gibson2.scenes.gibson_indoor_scene import StaticIndoorScene
from gibson2.objects.ycb_object import YCBObject
from gibson2.utils.utils import parse_config
from gibson2.render.mesh_renderer.mesh_renderer_settings import MeshRendererSettings
import numpy as np
from gibson2.render.profiler import Profiler
import gibson2
import os

INFO:root:Importing iGibson (gibson2 module)
INFO:root:Assets path: /opt/igibson/gibson2/data/assets
INFO:root:Gibson Dataset path: /opt/igibson/gibson2/data/g_dataset
INFO:root:iG Dataset path: /opt/igibson/gibson2/data/ig_dataset
INFO:root:3D-FRONT Dataset path: /opt/igibson/gibson2/data/threedfront_dataset
INFO:root:CubiCasa5K Dataset path: /opt/igibson/gibson2/data/cubicasa_dataset
INFO:root:Example path: /opt/igibson/gibson2/examples
INFO:root:Example config path: /opt/igibson/gibson2/examples/configs


torch is not available, falling back to rendering to memory(instead of tensor)


In [2]:
config = parse_config(os.path.join(gibson2.example_config_path, 'turtlebot_demo.yaml'))
settings = MeshRendererSettings(enable_shadow=False, msaa=False)
s = Simulator(mode='headless', image_width=256,
              image_height=256, rendering_settings=settings)

scene = StaticIndoorScene('Rs',
                          build_graph=True,
                          pybullet_load_texture=True)
s.import_scene(scene)
turtlebot = Turtlebot(config)
s.import_robot(turtlebot)

for _ in range(10):
    obj = YCBObject('003_cracker_box')
    s.import_object(obj)
    obj.set_position_orientation(np.random.uniform(
        low=0, high=2, size=3), [0, 0, 0, 1])

print(s.renderer.instances)


INFO:root:Device 0 is available for rendering
INFO:root:Using device 0 for rendering
INFO:root:IndoorScene model: Rs
INFO:root:StaticIndoorScene scene: Rs


******************PyBullet Logging Information:
PyBullet Logging Information******************


INFO:root:Loading traversable graph
INFO:root:Loading /opt/igibson/gibson2/data/g_dataset/Rs/mesh_z_up.obj
/opt/igibson/gibson2/render/mesh_renderer/mesh_renderer_cpu.py:461: RuntimeWarning: divide by zero encountered in true_divide
  delta_uv1[:, 1] * delta_uv2[:, 0])
/opt/igibson/gibson2/render/mesh_renderer/mesh_renderer_cpu.py:463: RuntimeWarning: invalid value encountered in multiply
  delta_pos2 * delta_uv1[:, 1][:, None]) * r[:, None]
/opt/igibson/gibson2/render/mesh_renderer/mesh_renderer_cpu.py:465: RuntimeWarning: invalid value encountered in multiply
  delta_pos1 * delta_uv2[:, 0][:, None]) * r[:, None]
INFO:root:Loading robot model file: turtlebot/turtlebot.urdf
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/kobuki_description/meshes/main_body.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/mjcf_primitives/cube.obj
INFO:root:Loading /opt/igibson/gibson2/data/assets/models/turtlebot/kobuki_description/meshes/wheel.obj
INFO:root:Loading /opt

[Instance(0) -> Object(0), Robot(1) -> Objects(1,2,3,3,4,5,6,6,6,6,6,6,7,8,8,8,8,9,10,10,10,10,11,11,12,13), Instance(2) -> Object(14), Instance(3) -> Object(14), Instance(4) -> Object(14), Instance(5) -> Object(14), Instance(6) -> Object(14), Instance(7) -> Object(14), Instance(8) -> Object(14), Instance(9) -> Object(14), Instance(10) -> Object(14), Instance(11) -> Object(14)]


In [10]:
rgbs = []
for i in range(100):
    with Profiler('Simulator step'):
        turtlebot.apply_action([0.1, 0.1])
        s.step()
        rgb = s.renderer.render_robot_cameras(modes=('rgb'))[0]
        rgbs.append(rgb)

Simulator step: 25.53 fps, 0.03917 seconds
Simulator step: 30.40 fps, 0.03290 seconds
Simulator step: 32.03 fps, 0.03122 seconds
Simulator step: 32.10 fps, 0.03115 seconds
Simulator step: 31.79 fps, 0.03145 seconds
Simulator step: 32.03 fps, 0.03122 seconds
Simulator step: 31.99 fps, 0.03126 seconds
Simulator step: 32.20 fps, 0.03105 seconds
Simulator step: 32.05 fps, 0.03120 seconds
Simulator step: 32.11 fps, 0.03114 seconds
Simulator step: 33.66 fps, 0.02971 seconds
Simulator step: 33.49 fps, 0.02986 seconds
Simulator step: 31.48 fps, 0.03176 seconds
Simulator step: 33.06 fps, 0.03025 seconds
Simulator step: 31.76 fps, 0.03149 seconds
Simulator step: 31.49 fps, 0.03176 seconds
Simulator step: 32.58 fps, 0.03069 seconds
Simulator step: 32.20 fps, 0.03106 seconds
Simulator step: 32.52 fps, 0.03075 seconds
Simulator step: 33.06 fps, 0.03025 seconds
Simulator step: 32.54 fps, 0.03073 seconds
Simulator step: 32.41 fps, 0.03085 seconds
Simulator step: 31.76 fps, 0.03148 seconds
Simulator s

In [31]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt
from PIL import Image
def show_frame(idx):
    display(Image.fromarray(((rgbs[idx][:,:,:3] * 255).astype(np.uint8))))

interactive_plot = interactive(show_frame, idx=(0,99))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=49, description='idx', max=99), Output(layout=Layout(height='350px'))), …